This notebook is intended to demonstrate how vessel segmentation methods of ITKTubeTK can be applied to multi-channel MRI (MRA + T1, T2, etc).

In [1]:
import itk
from itk import TubeTK as ttk

from itkwidgets import view

import numpy as np

In [2]:
ImageType = itk.Image[itk.F, 3]

im1 = itk.imread("data/mra.mha",itk.F)

In [3]:
res = ttk.ResampleImage[ImageType].New()
res.SetInput(im1)
res.SetMakeHighResIso(True)
res.Update()
im1iso = res.GetOutput()

In [4]:
xp1=[-3.9,-54.8,12.3]
xp2=[18.4,-16,-21]
xp3=[-46,-15,-13]

vSeg = ttk.SegmentTubes[ImageType].New()
vSeg.SetInput(im1iso)
vSeg.SetVerbose(False)
vSeg.SetRadiusInObjectSpace( 0.33 )
vSeg.SetMinRoundness(0.2)
vSeg.SetVerbose(True)
tube = vSeg.ExtractTubeInObjectSpace( xp1, 1 )
tube = vSeg.ExtractTubeInObjectSpace( xp2, 2 )
tube = vSeg.ExtractTubeInObjectSpace( xp3, 3 )
tubeG = vSeg.GetTubeGroup()

In [5]:
# Example of accessing data in a tube
p = list(map(lambda x: tube.GetPoint(x).GetPositionInObjectSpace(), range(tube.GetNumberOfPoints())))
r = list(map(lambda x: tube.GetPoint(x).GetIntensity(), range(tube.GetNumberOfPoints())))
print(r)

[0.359974376474655, 0.36011069195614315, 0.3603887059522363, 0.36067443601370697, 0.36096853695935327, 0.36127169494669376, 0.3614006871243034, 0.36172354578945826, 0.3620576987441207, 0.3624039553195158, 0.362763154659106, 0.3631360306878859, 0.3635232518564094, 0.36392531829275987, 0.364342064074381, 0.36477327720984004, 0.3652186989037603, 0.36567802286552875, 0.3661508948200595, 0.36663691236139845, 0.36713562728394644, 0.36764649691350243, 0.3681696982169162, 0.3687040193407126, 0.36924824153524755, 0.36980103982892343, 0.37036109131055034, 0.3709270779310389, 0.3714976764050699, 0.37207013876730366, 0.37264529894997883, 0.37322140229852263, 0.37379729600756506, 0.37437140913071987, 0.3749446339382242, 0.375515004167819, 0.3760823610589245, 0.37664657508029525, 0.37720754797759226, 0.3777652150262033, 0.3783194548079636, 0.3788699498189861, 0.37941712745029066, 0.3799610419744714, 0.3805017379161582, 0.38103962885652976, 0.3811763107077472, 0.38170759039720364, 0.382235842881042, 

In [6]:
TubesToImageFilterType = ttk.ConvertTubesToImage[ImageType]
tubesToImageFilter = TubesToImageFilterType.New()
tubesToImageFilter.SetUseRadius(True)
tubesToImageFilter.SetColorByRidgeness(True)
tubesToImageFilter.SetTemplateImage(im1)
tubesToImageFilter.SetInput(tubeG)
tubesToImageFilter.Update()

outputImage = tubesToImageFilter.GetOutput()
view(outputImage)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageF3; proxy o…

In [9]:
writer = itk.SpatialObjectWriter[3.New()
writer.SetFileName("vessels.tre")
writer.SetInput(tubeG)
writer.Update()

TypeError: in method 'itkSpatialObjectWriter2_SetInput', argument 2 of type 'itkSpatialObject2 *'